In [ ]:
import tensorflow as tf
import os

In [ ]:
!pip install tensorflow

In [ ]:
import cv2
import imghdr

In [ ]:
data_dir=os.path.join('dataset')

In [ ]:
os.listdir(data_dir)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
img = cv2.imread(os.path.join(data_dir, 'kulkas','kulkas (1).jpg'))

In [ ]:
img.shape

In [ ]:
for image_class in os.listdir(data_dir):
    print(f'total training {image_class} images:' , len(os.listdir(os.path.join(data_dir,image_class))))

In [ ]:
batch_size = 16
img_height = 224
img_width = 224

In [ ]:
val_path = 'split/train'
train_path = 'split/train'

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

In [ ]:
from tensorflow.keras import regularizers

In [ ]:
num_classes = len(class_names)

model = tf.keras.Sequential([

    tf.keras.layers.Conv2D(filters=32, kernel_size=(5,5), strides=(4,4), 
                           activation='relu', input_shape=(224,224,3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(5,5), strides=(1,1), 
                           activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    tf.keras.layers.Conv2D(filters=96, kernel_size=(3,3), strides=(1,1), 
                           activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    # tf.keras.layers.Conv2D(filters=96, kernel_size=(3,3), strides=(1,1), 
    #                        activation='relu', padding="same"),
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),

    tf.keras.layers.Dense(num_classes, activation='softmax'),
])

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

train_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip = True)


val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'split/train',
        target_size=(img_height, img_width),
        batch_size=batch_size)

validation_generator = val_datagen.flow_from_directory(
        'split/val',
        target_size=(img_height, img_width),
        batch_size=batch_size, shuffle=False)

In [ ]:
epochs=30
history = model.fit(
  train_generator,
  validation_data=validation_generator,
  epochs=epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
#menyimpan model

from tensorflow.keras.models import load_model 
model.save('model_v4.h5')